# Градиентный бустинг своими руками

**Внимание:** в тексте задания произошли изменения - поменялось число деревьев (теперь 50), правило изменения величины шага в задании 3 и добавился параметр `random_state` у решающего дерева. Правильные ответы не поменялись, но теперь их проще получить. Также исправлена опечатка в функции `gbm_predict`.

В этом задании будет использоваться датасет `boston` из `sklearn.datasets`. Оставьте последние 25% объектов для контроля качества, разделив `X` и `y` на `X_train`, `y_train` и `X_test`, `y_test`.

Целью задания будет реализовать простой вариант градиентного бустинга над регрессионными деревьями для случая квадратичной функции потерь.

In [15]:
from sklearn import ensemble, model_selection, metrics, datasets, tree

import numpy as np
import pandas as pd
import xgboost as xgb

In [2]:
boston = datasets.load_boston()
X, y = boston.data, boston.target

In [63]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((379, 13), (127, 13), (379,), (127,))

In [66]:
X_train, X_test, y_train, y_test = model_selection.train_test_split(boston.data, boston.target, 
                                                                                    test_size = 0.25,
                                                                                    random_state = 1)

## Задание 1

Как вы уже знаете из лекций, **бустинг** - это метод построения композиций базовых алгоритмов с помощью последовательного добавления к текущей композиции нового алгоритма с некоторым коэффициентом. 

Градиентный бустинг обучает каждый новый алгоритм так, чтобы он приближал антиградиент ошибки по ответам композиции на обучающей выборке. Аналогично минимизации функций методом градиентного спуска, в градиентном бустинге мы подправляем композицию, изменяя алгоритм в направлении антиградиента ошибки.

Воспользуйтесь формулой из лекций, задающей ответы на обучающей выборке, на которые нужно обучать новый алгоритм (фактически это лишь чуть более подробно расписанный градиент от ошибки), и получите частный ее случай, если функция потерь `L` - квадрат отклонения ответа композиции `a(x)` от правильного ответа `y` на данном `x`.

Если вы давно не считали производную самостоятельно, вам поможет таблица производных элементарных функций (которую несложно найти в интернете) и правило дифференцирования сложной функции. После дифференцирования квадрата у вас возникнет множитель 2 — т.к. нам все равно предстоит выбирать коэффициент, с которым будет добавлен новый базовый алгоритм, проигноируйте этот множитель при дальнейшем построении алгоритма.

In [76]:
def getS(b, y):
    return y - b    

## Задание 2

Заведите массив для объектов `DecisionTreeRegressor` (будем их использовать в качестве базовых алгоритмов) и для вещественных чисел (это будут коэффициенты перед базовыми алгоритмами). 

В цикле от обучите последовательно 50 решающих деревьев с параметрами `max_depth=5` и `random_state=42` (остальные параметры - по умолчанию). В бустинге зачастую используются сотни и тысячи деревьев, но мы ограничимся 50, чтобы алгоритм работал быстрее, и его было проще отлаживать (т.к. цель задания разобраться, как работает метод). Каждое дерево должно обучаться на одном и том же множестве объектов, но ответы, которые учится прогнозировать дерево, будут меняться в соответствие с полученным в задании 1 правилом. 

Попробуйте для начала всегда брать коэффициент равным 0.9. Обычно оправдано выбирать коэффициент значительно меньшим - порядка 0.05 или 0.1, но т.к. в нашем учебном примере на стандартном датасете будет всего 50 деревьев, возьмем для начала шаг побольше.

В процессе реализации обучения вам потребуется функция, которая будет вычислять прогноз построенной на данный момент композиции деревьев на выборке `X`:

```
def gbm_predict(X):
    return [sum([coeff * algo.predict([x])[0] for algo, coeff in zip(base_algorithms_list, coefficients_list)]) for x in X]
(считаем, что base_algorithms_list - список с базовыми алгоритмами, coefficients_list - список с коэффициентами перед алгоритмами)
```

Эта же функция поможет вам получить прогноз на контрольной выборке и оценить качество работы вашего алгоритма с помощью `mean_squared_error` в `sklearn.metrics`. 

Возведите результат в степень 0.5, чтобы получить `RMSE`. Полученное значение `RMSE` — **ответ в пункте 2**.

In [21]:
def gbm_predict(X):
    return [sum([coeff * algo.predict([x])[0] for algo, coeff in zip(base_algorithms_list, coefficients_list)]) for x in X]

In [ ]:
base_algorithms_list = [tree.DecisionTreeRegressor(max_depth = 5, random_state = 42)] * 50
coefficients_list = np.ones(50) * 0.9

In [36]:
X_test[0]
base_algorithms_list[i].predict([X_test[0]])[0]


32.127500000000005

In [50]:
base_algorithms_list[i].predict(X_test).shape
y_train.shape

(379,)

In [86]:
s_train = y_train
for i in np.arange(50):
    base_algorithms_list[i].fit(X_train, s_train)
    
for i in np.arange(50):
    base_algorithms_list[i].fit(X_train, s_train)
    #predict = base_algorithms_list[i].predict(X_train)
    predict = gbm_predict(X_train)
    s_train = getS(predict, s_train)
    print("Pred : ", predict[:10])
    #print("S : ", s_train[:10])
    
gbm_predict(X_train[:10])

Pred :  [940.0344827586199, 964.2162162162155, 1241.678571428572, 964.2162162162155, 964.2162162162155, 1101.3088235294113, 367.105263157895, 1241.678571428572, 1445.7374999999995, 605.3684210526318]
Pred :  [41361.51724137927, 42243.69924812021, 54633.85714285721, 42243.69924812021, 42243.69924812021, 48457.58823529407, 16152.6315789474, 54633.85714285721, 63612.44999999991, 26636.21052631583]
Pred :  [1819906.7586206861, 1858722.766917291, 2403889.7142857173, 1858722.766917291, 1858722.766917291, 2132133.8823529417, 710715.7894736856, 2403889.7142857173, 2798947.799999995, 1171993.2631578955]
Pred :  [80075897.37931018, 81783801.74436077, 105771147.42857149, 81783801.74436077, 81783801.74436077, 93813890.82352945, 31271494.736842204, 105771147.42857149, 123153703.1999998, 51567703.578947455]
Pred :  [3523339484.689643, 3598487276.7518754, 4653930486.857149, 3598487276.7518754, 3598487276.7518754, 4127811196.2352977, 1375945768.421055, 4653930486.857149, 5418762940.799995, 2268978957.

Pred :  [1.372271798852485e+62, 1.4015403936731584e+62, 1.8126148753719882e+62, 1.4015403936731584e+62, 1.4015403936731584e+62, 1.6077017046457502e+62, 5.35903957867106e+61, 1.8126148753719882e+62, 2.1105021530395527e+62, 8.837229137473052e+61]
Pred :  [6.037995914950941e+63, 6.1667777321618944e+63, 7.975505451636749e+63, 6.1667777321618944e+63, 6.1667777321618944e+63, 7.073887500441303e+63, 2.357977414615269e+63, 7.975505451636749e+63, 9.28620947337405e+63, 3.8883808204881414e+63]
Pred :  [2.656718202578412e+65, 2.713382202151233e+65, 3.509222398720172e+65, 2.713382202151233e+65, 2.713382202151233e+65, 3.112510500194172e+65, 1.0375100624307186e+65, 3.509222398720172e+65, 4.0859321682845803e+65, 1.7108875610147836e+65]
Pred :  [1.168956009134501e+67, 1.1938881689465416e+67, 1.544057855436876e+67, 1.1938881689465416e+67, 1.1938881689465416e+67, 1.3695046200854372e+67, 4.565044274695156e+66, 1.544057855436876e+67, 1.7978101540452162e+67, 7.527905268465045e+66]
Pred :  [5.143406440191804e

[3.179251850156378e+83,
 3.2470607450946373e+83,
 4.1994298804116655e+83,
 3.2470607450946373e+83,
 3.2470607450946373e+83,
 3.7246911459294964e+83,
 1.241571568387435e+83,
 4.1994298804116655e+83,
 4.889569165832096e+83,
 2.0473915669666347e+83]

In [22]:
coefficients_list.shape

(50,)

## Задание 3

Вас может также беспокоить, что двигаясь с постоянным шагом, вблизи минимума ошибки ответы на обучающей выборке меняются слишком резко, перескакивая через минимум. 

Попробуйте уменьшать вес перед каждым алгоритмом с каждой следующей итерацией по формуле `0.9 / (1.0 + i)`, где `i` - номер итерации (от 0 до 49). Используйте качество работы алгоритма как **ответ в пункте 3**. 

В реальности часто применяется следующая стратегия выбора шага: как только выбран алгоритм, подберем коэффициент перед ним численным методом оптимизации таким образом, чтобы отклонение от правильных ответов было минимальным. Мы не будем предлагать вам реализовать это для выполнения задания, но рекомендуем попробовать разобраться с такой стратегией и реализовать ее при случае для себя.

## Задание 4

Реализованный вами метод - градиентный бустинг над деревьями - очень популярен в машинном обучении. Он представлен как в самой библиотеке `sklearn`, так и в сторонней библиотеке `XGBoost`, которая имеет свой питоновский интерфейс. На практике `XGBoost` работает заметно лучше `GradientBoostingRegressor` из `sklearn`, но для этого задания вы можете использовать любую реализацию. 

Исследуйте, переобучается ли градиентный бустинг с ростом числа итераций (и подумайте, почему), а также с ростом глубины деревьев. На основе наблюдений выпишите через пробел номера правильных из приведенных ниже утверждений в порядке возрастания номера (это будет **ответ в п.4**):

    1. С увеличением числа деревьев, начиная с некоторого момента, качество работы градиентного бустинга не меняется существенно.

    2. С увеличением числа деревьев, начиная с некоторого момента, градиентный бустинг начинает переобучаться.

    3. С ростом глубины деревьев, начиная с некоторого момента, качество работы градиентного бустинга на тестовой выборке начинает ухудшаться.

    4. С ростом глубины деревьев, начиная с некоторого момента, качество работы градиентного бустинга перестает существенно изменяться

## Задание 5

Сравните получаемое с помощью градиентного бустинга качество с качеством работы линейной регрессии. 

Для этого обучите `LinearRegression` из `sklearn.linear_model` (с параметрами по умолчанию) на обучающей выборке и оцените для прогнозов полученного алгоритма на тестовой выборке `RMSE`. Полученное качество - ответ в **пункте 5**. 

В данном примере качество работы простой модели должно было оказаться хуже, но не стоит забывать, что так бывает не всегда. В заданиях к этому курсу вы еще встретите пример обратной ситуации.